# SageMaker JumpStart 모델 온보딩 (TGI)


## ML 모델 패키징 프로세스

<img src="images/ml-model-publishing-workflow.png"/>

다음 다이어그램은 ML 모델 패키징 프로세스의 개요를 제공합니다.


- **1단계** 모델 아티팩트 및 서빙/스코어링 로직 저장
- **2단계** 추론을 수행하는 SageMaker에서 모델을 호스팅하는 데 사용되는 컨테이너를 생성하고 ECR에 푸시
- **3단계** SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
- **4단계** ML 모델을 모델 패키지로 패키징
- **5단계** Amazon SageMaker에 배포하여 ML 모델 패키지 검증
- **6단계** AWS Marketplace에 ML 모델 등록

> **참고**: 모든 로컬 작업은 최적의 성능과 호환성을 위해 반드시 GPU가 지원되는 SageMaker Notebook 인스턴스에서 수행되어야 합니다.
> 
> [Deploy models with DJL Serving](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-djl-serving.html)

In [ ]:
install_needed = True
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker transformers huggingface_hub --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

# Start

In [ ]:
%load_ext autoreload
%autoreload 2

### Model Store

In [ ]:
import os
import time
import boto3
import logging

from pathlib import Path
import huggingface_hub

import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.pytorch.model import PyTorchModel

from sagemaker import get_execution_role
from sagemaker.session import Session

sagemaker_session = Session()

artifacts_bucket_name = sagemaker_session.default_bucket()

execution_role_arn = get_execution_role()
region = sagemaker_session.boto_region_name

os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/.cache'

#### https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html

In [ ]:
from sagemaker import instance_types

ref_model_id = "meta-textgeneration-llama-3-2-3b"
instance_type = instance_types.retrieve_default(
    model_id=ref_model_id,
    model_version="1.1.2",
    scope="inference")
print(instance_type)

<br>

## [**Step 1**] 모델 아티팩트 및 서빙/스코어링 로직 저장
---

In [ ]:
model_id='meta-llama/Llama-3.2-3B'

model_name = model_id.split("/")[-1].lower()
model_name = model_name.replace(".", "-")
model_name

In [ ]:
hf_local_download_dir = Path.cwd() / model_name
hf_local_download_dir.mkdir(exist_ok=True)

huggingface_hub.snapshot_download(
    repo_id=model_id,
    revision="main",
    local_dir=hf_local_download_dir,
    # use_auth_token="hf_your_access_token"
)

In [ ]:
!rm -rf shell && mkdir shell

In [ ]:
%%writefile shell/model_compression_upload.sh

cd llama-3-2-3b
tar cvf - * | pigz > model.tar.gz

cd ..
sudo rm -rf compressed_model && mkdir compressed_model
mv llama-3-2-3b/model.tar.gz compressed_model/

In [ ]:
!sh ./shell/model_compression_upload.sh

<br>

## [**Step 2**] 추론 수행을 위한 SageMaker 컨테이너 생성 및 ECR 등록
---

In [ ]:
# image_uri = get_huggingface_llm_image_uri(
#   backend="huggingface", # or lmi
#   region=region
# )
# image_uri
image_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.1.1-gpu-py311-cu124-ubuntu22.04-v2.0"

In [ ]:
account = sagemaker.Session().account_id()
ecr_image_uri = image_uri.replace("763104351884", account)
ecr_image_uri

In [ ]:
!rm -rf docker && mkdir docker

In [ ]:
%%writefile docker/sagemaker-entrypoint.sh
#!/bin/bash

if [[ -z "${HF_MODEL_ID}" ]]; then
  echo "HF_MODEL_ID must be set"
  exit 1
fi
export MODEL_ID="${HF_MODEL_ID}"

if [[ -n "${HF_MODEL_REVISION}" ]]; then
  export REVISION="${HF_MODEL_REVISION}"
fi

if [[ -n "${SM_NUM_GPUS}" ]]; then
    NUM_SHARD="${SM_NUM_GPUS}"
else
    NUM_SHARD=$(nvidia-smi --list-gpus | wc -l)
fi

export NUM_SHARD


if [[ -n "${HF_MODEL_QUANTIZE}" ]]; then
  export QUANTIZE="${HF_MODEL_QUANTIZE}"
fi

if [[ -n "${HF_MODEL_TRUST_REMOTE_CODE}" ]]; then
  export TRUST_REMOTE_CODE="${HF_MODEL_TRUST_REMOTE_CODE}"
fi

text-generation-launcher --port 8080


In [ ]:
%%writefile docker/Dockerfile

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.1.1-gpu-py311-cu124-ubuntu22.04-v2.0

ENV HF_MODEL_ID "/opt/ml/model"
ENV HF_MODEL_QUANTIZE "bitsandbytes"
ENV HF_MODEL_TRUST_REMOTE_CODE "true"
# ENV SM_NUM_GPUS "4"
RUN pip install --no-cache-dir numpy==1.24.3

COPY sagemaker-entrypoint.sh entrypoint.sh
RUN chmod +x entrypoint.sh

ENTRYPOINT ["./entrypoint.sh"]

In [ ]:
%%writefile docker/build_and_push.sh

original_image_uri="763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.1.1-gpu-py311-cu124-ubuntu22.04-v2.0"

algorithm_name="huggingface-pytorch-tgi-inference"

cd docker

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

target_image_uri="${account}.dkr.ecr.us-west-2.amazonaws.com/${algorithm_name}:2.6.0-tgi3.1.1-gpu-py311-cu124-ubuntu22.04-v2.0"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1


if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com"

# docker pull $original_image_uri
# docker image tag $original_image_uri $target_image_uri

docker build -f Dockerfile -t ${target_image_uri} .

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${target_image_uri}

docker push ${target_image_uri}

In [ ]:
!sh ./docker/build_and_push.sh > /dev/null 2>&1

<br>

## [**Step 3**] SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
---

SageMaker 호스팅 엔드포인트로 배포하기 전에 로컬 모드 엔드포인트로 배포할 수 있습니다. 로컬 모드는 현재 개발 중인 환경에서 도커 컨테이너를 실행하여 SageMaker 프로세싱/훈련/추론 작업을 에뮬레이트할 수 있습니다. 추론 작업의 경우는 Amazon ECR의 딥러닝 프레임워크 기반 추론 컨테이너를 로컬로 가져오고(docker pull) 컨테이너를 실행하여(docker run) 모델 서버를 시작합니다.


### SageMaker Endpoint (Local Mode)

로컬 모드는 필수로 수행할 필요는 없지만, 디버깅에 많은 도움이 됩니다. 또한, 로컬 모드 사용 시에는 모델을 S3에 반드시 업로드할 필요 없이 로컬 디렉터리에서도 로드할 수 있습니다. (`container` 변수 참조)

In [ ]:
import boto3
import time
import json


# Set to True to enable SageMaker to run locally
local_mode = True
# local_mode = False
if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_session.config = {'local': {'local_code': True}}
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/{model_name}"
else:
    instance_type = "ml.g5.12xlarge"
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = f"{compressed_model_path}/model.tar.gz"

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"{model_name}-{ts}"
endpoint_config_name = f"{model_name}-endpoint-config-{ts}"
endpoint_name = f"{model_name}-endpoint-{ts}"
model_data

print(f'--- SageMaker Model Name: {sm_model_name}')
print(f'--- Endpoint Config Name: {endpoint_config_name}')     
print(f'--- Endpoint Name: {endpoint_name}')
print(f'--- Model Data: {model_data}')

In [ ]:
# env_var = {
#     'HF_MODEL_ID': "/opt/ml/model",
#     'SM_NUM_GPUS':'4',
#     'HF_MODEL_QUANTIZE':'bitsandbytes',
#     'HF_MODEL_TRUST_REMOTE_CODE' : 'true'
# }

env_var = {
}

container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env_var
}

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=execution_role_arn, 
    PrimaryContainer=container,
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            'ModelDataDownloadTimeoutInSeconds': 300,
            'ContainerStartupHealthCheckTimeoutInSeconds': 300,
            
        },
    ],
)
#print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
!docker ps

In [ ]:
!docker kill 162db8246dcf

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

In [ ]:
!docker ps

### Inference Test

In [ ]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"

sample_input = {
    "inputs": prompt,
    "parameters": {
        "max_tokens":256,
        "top_p": 0.9,
        "temperature": 0.6,
        "max_tokens": 512,
        "stop": ["<|eot_id|>"]
    }
}

In [ ]:
%%time
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Accept="application/json",
    ContentType="application/json",
    Body=json.dumps(sample_input)
)
data = response["Body"].read()
output = json.loads(data)
output[0]['generated_text']

### Clean up

In [ ]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [ ]:
delete_endpoint(sm_client, endpoint_name)

<br>

## [**Step 3**] SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
---

SageMaker 호스팅 엔드포인트로 배포하기 전에 로컬 모드 엔드포인트로 배포할 수 있습니다. 로컬 모드는 현재 개발 중인 환경에서 도커 컨테이너를 실행하여 SageMaker 프로세싱/훈련/추론 작업을 에뮬레이트할 수 있습니다. 추론 작업의 경우는 Amazon ECR의 딥러닝 프레임워크 기반 추론 컨테이너를 로컬로 가져오고(docker pull) 컨테이너를 실행하여(docker run) 모델 서버를 시작합니다.


In [ ]:
compressed_model_path = f"s3://{artifacts_bucket_name}/{model_name}/compressed_model"
compressed_model_path

In [ ]:
!aws s3 sync ./compressed_model/ $compressed_model_path

### SageMaker Endpoint (Local Mode)

로컬 모드는 필수로 수행할 필요는 없지만, 디버깅에 많은 도움이 됩니다. 또한, 로컬 모드 사용 시에는 모델을 S3에 반드시 업로드할 필요 없이 로컬 디렉터리에서도 로드할 수 있습니다. (`container` 변수 참조)

In [ ]:
import boto3
import time
import json


# Set to True to enable SageMaker to run locally
local_mode = False

if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_session.config = {'local': {'local_code': True}}
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/{model_name}"
else:
    instance_type = "ml.g5.12xlarge" ###### instance type
    
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = f"{compressed_model_path}/model.tar.gz"

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"{model_name}-{ts}"
endpoint_config_name = f"{model_name}-endpoint-config-{ts}"
endpoint_name = f"{model_name}-endpoint-{ts}"

print(f'--- SageMaker Model Name: {sm_model_name}')
print(f'--- Endpoint Config Name: {endpoint_config_name}')     
print(f'--- Endpoint Name: {endpoint_name}')
print(f'--- Model Data: {model_data}')


In [ ]:
env_var = {}

container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env_var
}

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=execution_role_arn, 
    PrimaryContainer=container,
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            'ModelDataDownloadTimeoutInSeconds': 300,
            'ContainerStartupHealthCheckTimeoutInSeconds': 300
            
        },
    ]
)

print("Model Arn: " + create_model_response["ModelArn"])
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(30)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"

sample_input = {
    "inputs": prompt,
    "parameters": {
        "max_tokens":256,
        "top_p": 0.9,
        "temperature": 0.6,
        "max_tokens": 512,
        "stop": ["<|eot_id|>"]
    }
}

In [ ]:
%%time
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Accept="application/json",
    ContentType="application/json",
    Body=json.dumps(sample_input)
)
data = response["Body"].read()
output = json.loads(data)
output[0]['generated_text']

### Clean up

In [ ]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [ ]:
delete_endpoint(sm_client, endpoint_name)

<br>

## [**Step 4**] ML 모델을 모델 패키지로 패키징
---
이 **step**에서는 아티팩트(ECR 이미지 및 학습된 모델 아티팩트)를 ModelPackage로 패키징하는 방법을 살펴봅니다. 이 작업을 완료하면 AWS 마켓플레이스에서 제품을 사전 학습된 모델로 등록할 수 있습니다.

**Note:** 모델을 여러 하드웨어 유형(CPU/GPU/Inferentia)에 배포할 수 있는 경우, 일반적으로 사용되는 컨테이너 이미지가 각각 다르기 때문에 각각에 대해 모델패키지를 생성하고 MP 목록에 다른 버전으로 추가해야 합니다.  

### 모델 패키지 사전 준비
모델 패키지는 추론에 필요한 모든 요소를 패키지로 묶은 모델 아티팩트에 대한 재사용 가능한 추상화 형태입니다. 이는 모델 데이터 위치(선택 사항)와 함께 사용할 추론 이미지를 정의하는 추론 사양으로 구성됩니다. ModelPackage는 AWS 마켓플레이스에 판매자로 등록할 AWS 계정에서 생성해야 합니다.

In [ ]:
import os
import time
import boto3
import logging

from pathlib import Path
import huggingface_hub

import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.pytorch.model import PyTorchModel

from sagemaker import get_execution_role
from sagemaker.session import Session

sagemaker_session = Session()

artifacts_bucket_name = sagemaker_session.default_bucket()

execution_role_arn = get_execution_role()
region = sagemaker_session.boto_region_name

s3_client = sagemaker_session.boto_session.client("s3")
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_id='meta-llama/Llama-3.2-3B'

model_name = model_id.split("/")[-1].lower()
model_name = model_name.replace(".", "-")
model_name

In [ ]:
compressed_model_path = f"s3://{artifacts_bucket_name}/{model_name}/compressed_model"
compressed_model_path

In [ ]:
model_data = f"{compressed_model_path}/model.tar.gz"
model_data

In [ ]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)
account = sagemaker.Session().account_id()
ecr_image_uri = image_uri.replace("763104351884", account)
ecr_image_uri

### 모델 패키지 생성
모델 패키지 생성 프로세스에서는 다음을 지정해야 합니다:
  1. 도커 이미지
  2. 모델 아티팩트
    - tar.gz 형태로 압축된 모델 아티팩트가 제공되어야 합니다.
        
판매자(및 구매자)에게 Amazon SageMaker에서 제품이 작동한다는 확신을 주기 위해, AWS Marketplace에 제품을 리스팅하기 전에 SageMaker는 기본적인 유효성 검사를 위와 같이 진행하였습니다. 이 유효성 검사 프로세스가 성공해야만 제품을 AWS Marketplace에 리스팅할 수 있습니다. 이 유효성 검사 프로세스는 사용자가 제공한 유효성 검사 프로필과 샘플 데이터를 사용하여 모델을 사용하여 계정에서 변환 작업을 생성하여 추론 이미지가 SageMaker에서 작동하는지 확인합니다.

다음으로, ML 모델에 적합한 인스턴스 크기를 식별해야 하며, ML 모델 위에서 성능 테스트를 실행하여 이를 확인할 수 있습니다.

**Note:** 모델 튜닝 외에도 인스턴스 유형을 식별할 때 모델의 요구 사항을 고려해야 합니다.  모델이 GPU 리소스를 사용하지 않는 경우 GPU 인스턴스 유형을 포함하지 마세요. 마찬가지로 모델이 GPU 리소스를 사용하지만 단일 GPU만 사용할 수 있는 경우, 여러 개의 GPU가 있는 인스턴스 유형을 포함하지 마세요. 성능상의 이점은 없이 사용자의 인프라 요금만 증가시킬 수 있기 때문입니다.

### 테스트용 데이터 만들기

In [ ]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"

sample_input = {
    "inputs": prompt,
    "parameters": {
        "max_tokens":256,
        "top_p": 0.9,
        "temperature": 0.6,
        "max_tokens": 512,
        "stop": ["<|eot_id|>"]
    }
}

In [ ]:
import json
json_line = json.dumps(sample_input)
s3_client.put_object(Bucket=artifacts_bucket_name, Key=f"{model_name}/validation-input-json/input.jsonl", Body=json_line)

In [ ]:
validation_file_name = "input.jsonl"
validation_input_path = f"s3://{artifacts_bucket_name}/{model_name}/validation-input-json/"
validation_output_path = f"s3://{artifacts_bucket_name}/{model_name}/validation-output-jsonl/"
validation_input_path

### 패키지 생성

In [ ]:
instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"{model_name}-{ts}"

print(f'--- SageMaker Model Name: {sm_model_name}')

# Define parameters
model_description = "marketplace-model-test" #"<<YourModelDescription>>"

# <<YourSupportedContentTypes>>
supported_content_types = ["application/json"] #["text/csv", "application/json", "application/json", "application/jsonlines"]

# <<YourSupportedResponseMIMETypes>>
supported_response_MIME_types = [ 
    "application/json",
]

supported_realtime_inference_instance_types = ["ml.g5.2xlarge", "ml.g5.4xlarge", "ml.g5.12xlarge", "ml.g5.16xlarge", "ml.g5.24xlarge","ml.g5.48xlarge"]
supported_batch_transform_instance_types = ["ml.g5.xlarge"] #  Don't use batch transform. And, the Batch Transform validation step is not required

In [ ]:
model_package = sagemaker_session.sagemaker_client.create_model_package(
    ModelPackageName=sm_model_name,
    ModelPackageDescription=model_description,
    InferenceSpecification={
        "Containers": [
            {
                "Image": ecr_image_uri,
                "ModelDataUrl": model_data
            }
        ],
        "SupportedTransformInstanceTypes": supported_batch_transform_instance_types,
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_instance_types,
        "SupportedContentTypes": supported_content_types,
        "SupportedResponseMIMETypes": supported_response_MIME_types,
    },
    CertifyForMarketplace=True,  # Make sure to set this to True
   ValidationSpecification={
        'ValidationRole': execution_role_arn,
        'ValidationProfiles': [
            {
                'ProfileName': "validation",
                'TransformJobDefinition': {
                    'MaxConcurrentTransforms': 1,
                    'MaxPayloadInMB': 64,
                    'BatchStrategy': 'SingleRecord',
                    'TransformInput': {
                        'DataSource': {
                            'S3DataSource': {
                                'S3DataType': 'S3Prefix',
                                'S3Uri': f'{validation_input_path}input.jsonl'
                            }
                        },
                        'ContentType': 'application/json',
                        'CompressionType': 'None',
                        'SplitType': 'None'
                    },
                    'TransformOutput': {
                        'S3OutputPath': f'{validation_output_path}output.json',
                        'Accept': 'application/json',
                        'AssembleWith': 'None',
                    },
                    'TransformResources': {
                        'InstanceType': supported_batch_transform_instance_types[0],
                        'InstanceCount': 1,
                    }
                }
            },
        ]
    },
)

In [ ]:
model_package_list = []
sm_client = boto3.client("sagemaker")
model_list_pack = sm_client.list_model_packages()
model_package_list = model_list_pack['ModelPackageSummaryList']
NextToken = model_list_pack.get('NextToken')

while True:
    if model_list_pack.get('NextToken'):
        NextToken = model_list_pack.get('NextToken')
        model_list_pack = sm_client.list_model_packages(NextToken=NextToken)
        model_package_list.extend(model_list_pack['ModelPackageSummaryList'])
    else:
        break

model_package_list 

In [ ]:
# ModelPackageName='meta-llama-3-1-8b-instruct-2024-08-04-23-52-19'
ModelPackageName = sm_model_name
sm_client.describe_model_package(ModelPackageName=ModelPackageName)['ModelPackageStatusDetails']
# sm_client.delete_model_package(ModelPackageName=ModelPackageName)

In [ ]:
# sagemaker_session.wait_for_model_package(model_package_name=sm_model_name) # If failure occurs navigate to SageMaker Console > My marketplace model packages > select the failed ModelPackage for details. 

다음을 실행하기 전에, [Model Packages console from Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources)을 열어서 모델 생성의 성공했는지를 확인해야 합니다.
모델을 선택하고 **Validation** 탭을 열어서 validation 결과를 확인할 수 있습니다.

<br>

## [**Step 5**] Amazon SageMaker에 배포하여 ML 모델 패키지 검증
---

##### 모델 패키지에서 모델 객체 생성

#### SageMaker 모델을 Endpoint로 배포

In [ ]:
from sagemaker import ModelPackage

model = ModelPackage(
    role=execution_role_arn,
    model_package_arn=model_package["ModelPackageArn"],
    # model_package_arn="arn:aws:sagemaker:us-west-2:322537213286:model-package/meta-llama-3-1-8b-instruct-2024-08-04-04-24-21",
    sagemaker_session=sagemaker_session
)

In [ ]:
model.deploy(
    initial_instance_count=1,
    # instance_type=supported_realtime_inference_instance_types[0],
    instance_type='ml.g5.2xlarge',
    endpoint_name=sm_model_name,
    model_data_download_timeout=600,
    container_startup_health_check_timeout=300,
)
model.endpoint_name

#### boto3로 예시 호출

In [ ]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"

sample_input = {
    "inputs": prompt,
    "parameters": {
        "max_tokens":256,
        "top_p": 0.9,
        "temperature": 0.6,
        "stop": ["<|eot_id|>"]
    }
}

In [ ]:
%%time
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(sample_input),
)

json.load(response["Body"])

#### 생성된 endpoint configuration 과 endpoint 정리 

In [ ]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)

- 이 모델은 필수가 아니므로 삭제해도 됩니다. 
- 배포 가능한 모델을 삭제한다는 점에 유의하세요. 
- 모델 패키지는 삭제하지 않습니다.

In [ ]:
model.delete_model()

##### AWS 마켓플레이스에 모델을 게시하려면 모델 패키지 ARN을 지정해야 합니다. 다음 모델 패키지 ARN을 복사합니다. 

In [ ]:
model_package["ModelPackageArn"]

<br>

## [**Step 6**] AWS Marketplace에 ML 모델 등록
---

1.  모델 파트너는 AWS 마켓플레이스에서 [public profile](https://docs.aws.amazon.com/marketplace/latest/userguide/seller-registration-process.html#seller-public-profile)을 생성하고 seller로 등록합니다.
마켓플레이스의 상품은 무료 상품으로 등록되므로 세금 정보를 제공할 필요가 없습니다.

2. 세이지메이커 콘솔의 [Model Packages](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) 섹션에서 이 노트북에서 생성한 엔티티를 찾을 수 있습니다. 성공적으로 생성되고 유효성이 검사되었다면 해당 엔티티를 선택하고 **Publish new ML Marketplace listing**를 선택할 수 있을 것입니다.

<img src="images/publish-to-marketplace-action.png"/>

리스팅을 작성할 수 있는 [AWS Marketplace Management portal](https://aws.amazon.com/marketplace/management/ml-products/)로 리디렉션됩니다.

<img src="images/listing.png"/>

1. 모델이 여러 하드웨어 유형을 대상으로 하는 경우 각 ModelPackage를 별도의 버전으로 목록에 추가하는 것을 잊지 마세요.
2. 추가를 클릭하고 모델 정보를 입력합니다. Product visibility을 'Public'로 설정해야 합니다.

<img src="images/public.png"/>

3. 테스트를 진행할 account 에 대해 모델 접근을 위한 Allowlist에 추가합니다. 예) account `171503325295`, `572320329544` and `559110549532` for access to the model. 
For region support select: `us-east-1, us-west-2, eu-west-1, eu-central-1, eu-west-2, ap-northeast-1, ap-south-1, ca-central-1, us-east-2, ap-northeast-2`
<img src="images/allowlist-accs.png"/>

4. Pricing and terms 하에 pricing 모델을 설정합니다.
**Inference based pricing (custom metering) at $0**

(선택 사항) 컨테이너가 아래를 구현하지 않은 경우 이를 확인하고 다음을 진행하세요. 

```
I confirm that my model package supports the response header for custom metering. Example response header: X-Amzn-Inference-Metering:
{"Dimension": "inference.count", "ConsumedUnits": 3}
I understand that in absence of this header, default metering will be used instead.
```

<img src="images/inference-based-pricing.png"/>

5. Listing 상태는 다음과 같이 표시되어야 합니다:
**Do not click Sign off and publish**

<img src="images/status-1.png"/>

6. Vissibility status of the listing should be `Limited`.

<img src="images/status-2.png"/>




**Resources**
* [Publishing your product in AWS Marketplace](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-publishing-your-product-in-aws-marketplace.html)


https://medium.com/@aliasghar.arabi/deploy-llama3-on-aws-inferentia-using-sagemaker-lmi-and-djl-serving-aa241db17aa3